# Ejercicio 11: Web Scraping
# Nombre: Nelson Casa

## Objetivo de la práctica

El objetivo de este ejercicio es construir un web scraper que recoja datos de un website.

### Parte 0: Planificar
1. Identificar los datos que quieres obtener.
2. Elegir el sitio web objetivo.
3. Planificar la estructura del corpus.

Parte 0: Planificación del Web Scraper

**Datos a obtener:**
- Título de la receta
- Descripción
- Ingredientes
- Instrucciones
- Información nutricional
- Enlaces a recetas relacionadas

**Sitio web objetivo:**
- allrecipes.com (archivo HTML descargado localmente)

**Estructura del corpus:**
- Cada receta será un documento con campos:
  - title
  - description
  - ingredients
  - instructions
  - nutrition_facts
  - url

## Parte 1: Entender el sitio web objetivo

- Analizar la estructura de la página web a ser analizada.
- Identificar los elementos HTML que contienen los datos bsuscados.

### Carga y parseo del archivo HTML

En esta celda se carga el archivo HTML de una receta almacenado localmente y se crea un objeto `BeautifulSoup`, el cual permite analizar y navegar la estructura del documento HTML para realizar el proceso de scraping.

In [13]:
from bs4 import BeautifulSoup

file = '/content/Unstuffed Cabbage Roll Recipe.html'

# Load the HTML file
with open(file, "r", encoding="utf-8") as file:
    html_content = file.read()

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

### Extracción del título de la receta

En esta celda se localiza y extrae el título de la receta desde la etiqueta `<title>` del documento HTML, con el fin de identificar el nombre del contenido analizado.

In [14]:
title = soup.find("title")
title.string

'Unstuffed Cabbage Roll Recipe'

### Extracción de los ingredientes

En esta celda se identifican y extraen los ingredientes de la receta a partir de los elementos HTML que contienen la lista estructurada de ingredientes, limpiando el texto obtenido para su correcta visualización.


In [15]:
ingredients_section = soup.find_all("li", class_="mm-recipes-structured-ingredients__list-item")
for ingredient in ingredients_section:
    print(ingredient.text.strip())

2 pounds ground beef
1 large onion, chopped
1 small head cabbage, chopped
2 (14.5 ounce) cans diced tomatoes
1 (8 ounce) can tomato sauce
½ cup water
2 cloves garlic, minced
2 teaspoons salt
1 teaspoon ground black pepper


## Parte 2: Obtener los datos deseados

* Buscar dentro del contenido HTML y extraer la información.

### Extracción y validación de la información de la receta

En esta celda se extraen de forma estructurada los principales componentes de la receta: descripción, ingredientes, instrucciones e información nutricional.  
Posteriormente, los datos obtenidos se imprimen en pantalla para validar que el proceso de scraping se realizó correctamente.


La estructura del sitio no separa claramente instrucciones, por lo que se realiza una extracción basada en bloques de texto.

In [16]:
# Extracting the description
description = soup.find("meta", {"name": "description"})["content"]

# Extracting the ingredients
ingredients_section = soup.find_all("li", class_="mm-recipes-structured-ingredients__list-item")
ingredients = [ingredient.get_text().strip() for ingredient in ingredients_section]

# Extracting the instructions
instructions_section = soup.select("li.comp.mntl-sc-block-group--LI p")
instructions = [step.get_text(strip=True) for step in instructions_section]

# Extracting the nutrition information
nutrition_section = soup.find_all("span", class_="mm-recipes-nutrition-facts-label__nutrient-name mm-recipes-nutrition-facts-label__nutrient-name--has-postfix")
nutrition_facts = [fact.parent.get_text().strip().replace('\n', ' ') for fact in nutrition_section]

# Print the extracted information
print("Recipe Title:", title)
print("Description:", description)
print("Ingredients:")
for ingredient in ingredients:
    print("-", ingredient)
print("Instructions:")
for i, instruction in enumerate(instructions, 1):
    print(f"{i}. {instruction}")
print("Nutrition Facts:")
for fact in nutrition_facts:
    print("-", fact)


Recipe Title: <title>Unstuffed Cabbage Roll Recipe</title>
Description: Unstuffed cabbage rolls with ground beef, cabbage, garlic, and tomatoes make a family-pleasing comforting casserole that's perfect for weeknights.
Ingredients:
- 2 pounds ground beef
- 1 large onion, chopped
- 1 small head cabbage, chopped
- 2 (14.5 ounce) cans diced tomatoes
- 1 (8 ounce) can tomato sauce
- ½ cup water
- 2 cloves garlic, minced
- 2 teaspoons salt
- 1 teaspoon ground black pepper
Instructions:
1. Gather all ingredients.
2. Photographer: Kyle Carpenter / Food Styling: Julian Hensarling / Prop Styling: Prissy
3. Heat a Dutch oven or large skillet over medium-high heat. Cook and stir beef and onion in the hot Dutch oven until browned and crumbly, 5 to 7 minutes; drain and discard grease.
4. Photographer: Kyle Carpenter / Food Styling: Julian Hensarling / Prop Styling: Prissy
5. Add cabbage, tomatoes, tomato sauce, water, garlic, salt, and pepper and bring to a boil. Cover Dutch oven, reduce heat, and 

## Parte 3: Obtener enlaces relacionados
* Encontrar links a otras recetas para completar el corpus

### Identificación y filtrado de enlaces a recetas relacionadas

En esta celda se extraen todos los enlaces presentes en la página HTML y se filtran aquellos que corresponden a otras recetas.  
Los enlaces duplicados son eliminados con el fin de obtener un conjunto único de URLs que permita ampliar el corpus de recetas.


In [17]:
# Find all the links to other recipes
recipe_links = soup.find_all("a", href=True)

# Filter and print only the links that are likely to be recipes
recipe_urls = []
for link in recipe_links:
    href = link['href']
    if "recipe" in href:
        recipe_urls.append(href)


recipe_urls = list(set(recipe_urls))  # eliminar duplicados

# Print the recipe URLs
print("Linked Recipes:")
for url in recipe_urls:
    print(url)

Linked Recipes:
https://www.allrecipes.com/about-us-6648102
https://www.allrecipes.com/recipes/1947/everyday-cooking/quick-and-easy/
https://www.allrecipes.com/recipes/722/world-cuisine/european/german/
https://www.allrecipes.com/cook/4774659
https://www.allrecipes.com/recipes/77/drinks/
/about-allrecipes-test-kitchen-7550983
https://www.allrecipes.com/recipe/256611/paleo-stuffed-cabbage/
https://www.allrecipes.com/recipe/236131/spicy-unstuffed-cabbage/
https://www.allrecipes.com/recipes/1419/holidays-and-events/big-game/
https://www.allrecipes.com/recipes/78/breakfast-and-brunch/
https://www.allrecipes.com/recipes/723/world-cuisine/european/italian/
https://www.allrecipes.com/recipes/88/bbq-grilling/
https://www.allrecipes.com/cuisine-a-z-6740455
https://www.allrecipes.com/recipes/95/pasta-and-noodles/
https://www.allrecipes.com/allstars-8661275
https://www.allrecipes.com/recipes/16099/everyday-cooking/comfort-food/
https://www.allrecipes.com/recipes/695/world-cuisine/asian/chinese/
h

## Parte 4: Hacer RAG con las recetas obtenidas
* Una vez que se ha construido el corpus, implementar y desplegar RAG para realizar búsquedas en el corpus

Arquitectura del RAG

Web Scraping → Corpus → Embeddings → Búsqueda semántica → Respuesta

### Creación de la estructura del documento

En esta celda se organiza la información extraída de la receta en una estructura tipo diccionario, la cual representa un documento individual que formará parte del corpus para el sistema de recuperación de información.


In [18]:
documents = []

documents.append({
    "title": title,
    "description": description,
    "ingredients": ingredients,
    "instructions": instructions,
    "nutrition": nutrition_facts
})

### Construcción del documento de texto para el corpus

En esta celda se transforma la información estructurada de la receta en un único documento de texto.  
Este formato facilita su uso posterior en el proceso de vectorización y recuperación semántica dentro del sistema RAG.


In [19]:
documents = []

text = f"""
Recipe: Unstuffed Cabbage Roll
Ingredients: {', '.join(ingredients)}
Instructions: {' '.join(instructions)}
Nutrition: {', '.join(nutrition_facts)}
"""

documents.append(text)

Generar embeddings

Usamos Sentence Transformers

In [20]:
!pip install sentence-transformers

In [21]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

doc_embeddings = model.encode(documents)

Query semántica

In [22]:
query = "receta con carne molida y repollo"
query_embedding = model.encode([query])[0]

Similaridad coseno

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(
    [query_embedding],
    doc_embeddings
)

best_doc_index = np.argmax(similarities)
print("Documento más relevante:")
print(documents[best_doc_index])

Documento más relevante:

Recipe: Unstuffed Cabbage Roll
Ingredients: 2 pounds ground beef, 1 large onion, chopped, 1 small head cabbage, chopped, 2 (14.5 ounce) cans diced tomatoes, 1 (8 ounce) can tomato sauce, ½ cup water, 2 cloves garlic, minced, 2 teaspoons salt, 1 teaspoon ground black pepper
Instructions: Gather all ingredients. Photographer: Kyle Carpenter / Food Styling: Julian Hensarling / Prop Styling: Prissy Heat a Dutch oven or large skillet over medium-high heat. Cook and stir beef and onion in the hot Dutch oven until browned and crumbly, 5 to 7 minutes; drain and discard grease. Photographer: Kyle Carpenter / Food Styling: Julian Hensarling / Prop Styling: Prissy Add cabbage, tomatoes, tomato sauce, water, garlic, salt, and pepper and bring to a boil. Cover Dutch oven, reduce heat, and simmer until cabbage is tender, about 30 minutes. Photographer: Kyle Carpenter / Food Styling: Julian Hensarling / Prop Styling: Prissy Serve hot and enjoy! Photographer: Kyle Carpenter /